<a href="https://colab.research.google.com/github/Deep-Dey1/MISTRAL-RAG/blob/main/mistral_rag_infrencing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install torch transformers datasets faiss-cpu sentence-transformers wandb flask bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [17]:
import torch
torch.cuda.empty_cache()


In [16]:
import gc
gc.collect()
torch.cuda.empty_cache()


In [18]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load trained model and tokenizer
model_path = "deep0210/mistral-rag"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto")

# Ensure PAD token is set
tokenizer.pad_token = tokenizer.eos_token  # Helps with attention_mask

# Function to generate response
def generate_response(query):
    input_text = f"###Human: {query} ###Assistant:"
    input_ids = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True).input_ids.to(model.device)

    with torch.no_grad():
        output = model.generate(
            input_ids,
            max_length=128,
            pad_token_id=tokenizer.eos_token_id,  # Avoids infinite loops
            eos_token_id=tokenizer.eos_token_id,  # Ensures it stops properly
            do_sample=True,  # Enables randomness in responses
            temperature=0.7,  # Adjusts creativity (lower = more deterministic)
            top_p=0.9,  # Nucleus sampling
        )

    response = tokenizer.decode(output[0], skip_special_tokens=True)

    # Extract only the assistant's response (remove prompt from output)
    response = response.replace(input_text, "").strip()
    return response

# Example query
query = "Tell me a short story about a brave knight."
input_ids = tokenizer(query, return_tensors="pt").input_ids.to(model.device)
output = model.generate(input_ids, max_length=200, temperature=0.7, top_p=0.9)
response = tokenizer.decode(output[0], skip_special_tokens=True)

print(response)


Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-

Tell me a short story about a brave knight.

Once upon a time, there was a brave knight who was on a quest to find a magical sword. He had heard that the sword was hidden in a deep, dark cave, and he was determined to find it.

The knight set out on his journey, and after many days of traveling, he finally reached the cave. He entered the cave and began to explore, but he soon realized that he was not alone. There were many monsters and creatures living in the cave, and they were not happy to see him.

The knight fought bravely against the monsters, using his sword and shield to defend himself. He was injured in the battle, but he refused to give up. He continued to fight, and eventually he was able to defeat the monsters and claim the magical sword.

The knight was overjoyed to have found the sword, and he knew that
